# Mining Massive Datasets Problem Set 10

Ruben Hartenstein, Taha Erkoc

# Exercise 1

### a) Advantage and Disadvantage of a Smaller Damping factor β

*"This means Google can dictate the rate of convergence according to how small α is chosen to be. Consequently, Google engineers are forced to perform a delicate balancing act. The smaller α is, the faster the convergence, but the smaller α is, the less the true hyperlink structure of the web is used to determine web page importance. And slightly different values for α can produce very different PageRanks. Moreover, as α → 1, not only does convergence slow drastically, but sensitivity issues begin to surface as well" (Section 5.1 Rate of Convergence)*

A smaller damping factor results in faster convergence of the power iteration method. This is because the subdominant eigenvalue of the matrix scales with $\beta$ and a smaller $\beta$ leads to fewer iterations needed to compute the PageRank vector. This is crucial when working with extremely large matrices such as those encountered in web-scale computations. However, a smaller damping factor also reduces the influence of the actual hyperlink structure of the web, meaning that rankings rely less on the link connectivity rather than the teleportation component, thus making rankings less reflective of the "real" importance of web pages.


### b) Reducing Iterations by Adjusting Tolerance
*"Thus, a rough estimate of the number of iterations needed to converge to a tolerance level τ (measured by the residual,
$x^{(k)T} P − x^{(k)T} = x^{(k+1)T} − x^{(k)T}$)is $\frac{\log 10τ}{\log 10α}$" (Section 5.1 Rate of Convergence)*

The relationship between the number of iterations (k) and tolerance is shown by:

$k = \frac{\log (τ)}{\log (α)}$

or

$$
k = \frac{\log(\tau)}{\log(\lambda_2)}
$$

where:
- $\tau$ is the fixed tolerance level,
- $\lambda_2 = \beta \cdot \lambda_2(M)$, the second-largest (subdominant) eigenvalue,
- $\beta$ is the damping factor (called α in the paper).


To reduce the number of iterations to 100, we need to reduce the dampening factor $\beta$


$\frac{k_{old}}{k_{new}} = \frac{\log(\lambda_{2, old})}{\log(\lambda_{2, new})}$

Substituting the values:

$$
\frac{1000}{100} = \frac{\log(\lambda_{2, \text{old}})}{\log(\lambda_{2, \text{new}})}.
$$

Since $\lambda_2 = \beta \cdot \lambda_2(M)$:

$$
\log(\beta_{\text{old}} \cdot \lambda_2(M)) = 10 \cdot \log(\beta_{\text{new}} \cdot \lambda_2(M)).
$$

Simplify by canceling $\log(\lambda_2(M))$:

$$
\log(\beta_{\text{old}}) = 10 \cdot \log(\beta_{\text{new}})
$$

Solving for $\beta_{\text{new}}$:

$$
\beta_{\text{old}} = (\beta_{\text{new}})^{10}.
$$

Take the 10th root:

$$
\beta_{\text{new}} = (\beta_{\text{old}})^{1/10}.
$$

#### Example:
For $\beta_{\text{old}} = 0.85$:

$$
\beta_{\text{new}} = (0.85)^{1/10} \approx 0.984.
$$


### c) Teleports and Adjustments to the Google Matrix
*"One of the first modifications to the basic PageRank model suggested by its founders was a change to the teleportation matrix $E$. Rather than using $\frac{1}{n}ee^T$, they used $ev^T$,where $v^T > 0$ is a probability vector called the personalization
or teleportation vector. Since $v^T$ is a probability vector with positive elements,
every node is still directly connected to every other node, thus, $P$ is irreducible.
Using $v^T$ in place of $\frac{1}{n}e^T$ means that the teleportation probabilities are no longer uniformly distributed. Instead, each time a surfer teleports, he or she follows the probability distribution given in $v^T$ to jump to the next page(...)To produce a PageRank that is personalized for a particular user, only the constant vector $v^T$ added at each iteration must be modified" (Section 6.2)*

One way Google exploits this is for __spam reduction__: by altering the teleportation vector, Google can penalize pages associated with link farms or spam, reducing their PageRank and overall influence in the ranking.


They also planned to use this for __personalization__, with vector $v^T$ allowing for different PageRank vectors to be tailored to different user preferences or categories, such as sports or news.

# Exercise 2

### a)

In a clique, every node is connected to every other node which means that all nodes have the same in- and out-degree. Thus each node contributes and receives the same fraction of its PageRank. This symmetric structure allows for the flow of the PageRank to be uniform across all nodes in the clique.

### b)

The general form of matrix A is as follows:

$A = \beta * M + (1-\beta) * [\frac{1}{N}]_{NXN}$

When a node is a dead-end (has no outgoing links), its corresponding column in $M$ contains all zeros. To make $M$ a valid stochastic matrix, we replace the entire column of the dead-end nodes with $1/N$, saying there is a uniform probability to any other node. Doing this our matrix $M$ remains column-stochastic (each column sums up to $1$) and random teleport links are followed with a probability $1.0$ from dead-ends.

With this approach, the teleportation is already incorperated in our new matrix $M$ and our teleportation matrix $(1-\beta) * [\frac{1}{N}]_{NXN}$ no longer needs to adress the dead-end issue. It only serves as a random jump mechanism across all nodes.

### c) (in the PDF)


### d)

With random teleports we give the surfer a probability of $1 - \beta$ of jumping to a random page outside the spider trap rather than following links within the trap. This prevents the pagerank score from being permanently trapped.

For Dead-ends, with random teleports a surfer at a dead-end is assumed to teleport to any other page in the graph with an equal probability of $1/N$. For our formula this means that we replace the dead-end column in $M$ with uniform probabilities $1/N$, restoring the column-stochastic property. This ensures our matrix $M$ remanins valid for the PageRank calculation the algorithm converges to a meaningful result regardless of the graph structure.

# Exercise 3

In [1]:
import numpy as np

def pagerank(M, beta, epsilon):
    """
    Compute the PageRank vector using the Google formulation and the Power Iteration method.

    Parameters:
        M (numpy.ndarray): The dense adjacency matrix of the graph (column-stochastic).
        beta (float): The teleportation factor.
        epsilon (float): The convergence threshold.

    Returns:
        numpy.ndarray: The PageRank vector.
    """
    # Number of nodes in the graph
    N = M.shape[0]

    # Compute Google Matrix A
    teleportation_matrix = np.ones((N, N)) / N
    A = beta * M + (1 - beta) * teleportation_matrix

    # Initialize PageRank vector (uniform distribution)
    r = np.ones(N) / N

    # Power Iteration Method
    while True:
        r_new = A @ r
        # Check for convergence using L1 norm
        if np.linalg.norm(r_new - r, 1) < epsilon:
            break
        r = r_new

    return r

In [2]:
# Example matrix from exercise 1
M = np.array([
    [1/3, 1/2, 0],
    [1/3, 0, 1/2],
    [1/3, 1/2, 1/2]
])

# Parameters
beta = 1  # Teleportation factor
epsilon = 1/12  # Convergence threshold

# Compute PageRank
pagerank_vector = pagerank(M, beta, epsilon)
print("PageRank Vector:", pagerank_vector)

PageRank Vector: [0.23148148 0.31481481 0.4537037 ]


In [3]:
def generate_clique_matrix(n):
    """
    Generate the column-stochastic adjacency matrix for a clique graph with n vertices.

    Parameters:
        n (int): Number of vertices in the clique.

    Returns:
        numpy.ndarray: Column-stochastic adjacency matrix for the clique.
    """
    M = np.ones((n, n))  # Fully connected graph (all entries are 1)
    np.fill_diagonal(M, 0)  # Remove self-loops
    M /= M.sum(axis=0)  # Normalize to make it column-stochastic
    return M

In [4]:
# Generate M(4) and M(6)
M4 = generate_clique_matrix(4)
M6 = generate_clique_matrix(6)

# Compute PageRank vectors
pagerank_M4 = pagerank(M4, beta, epsilon)
pagerank_M6 = pagerank(M6, beta, epsilon)

print("PageRank Vector for M(4):", pagerank_M4)
print("PageRank Vector for M(6):", pagerank_M6)

PageRank Vector for M(4): [0.25 0.25 0.25 0.25]
PageRank Vector for M(6): [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]


# Exercise 4

In [5]:
def parse_graph(file_path):
    """
    Parses the Stanford web graph dataset and constructs dictionaries for in-neighbors and out-neighbors.
    
    Parameters:
        file_path (str): Path to the dataset file.
    
    Returns:
        dict: in_neighbors - Maps each node to a list of its in-neighbors.
        dict: out_neighbors - Maps each node to a list of its out-neighbors.
    """
    in_neighbors = {}
    out_neighbors = {}

    with open(file_path, 'r') as f:
        for line in f:
            # Skip comments
            if line.startswith("#"):
                continue
            
            # Parse the edge
            from_node, to_node = map(int, line.strip().split('\t'))
            
            # Update out-neighbors
            if from_node not in out_neighbors:
                out_neighbors[from_node] = []
            out_neighbors[from_node].append(to_node)
            
            # Update in-neighbors
            if to_node not in in_neighbors:
                in_neighbors[to_node] = []
            in_neighbors[to_node].append(from_node)
    
    return in_neighbors, out_neighbors

# Example usage
file_path = 'web-Stanford_small.txt'
in_neighbors, out_neighbors = parse_graph(file_path)

# Display a portion of the data
print("In-Neighbors:", dict(list(in_neighbors.items())[:5]))
print("Out-Neighbors:", dict(list(out_neighbors.items())[:5]))


In-Neighbors: {6548: [1], 15409: [1], 57031: [6548], 13102: [15409], 17794: [2]}
Out-Neighbors: {1: [6548, 15409], 6548: [57031], 15409: [13102], 2: [17794, 25202, 53625, 54582, 64930, 73764, 84477, 98628, 100193, 102355, 105318, 105730, 115926, 140864, 163550, 164599, 175799, 178642, 181714, 190453, 204189, 204604, 210870, 213966, 225119, 241596, 243294, 246897, 251658, 252915, 280935], 252915: [2]}


In [6]:
from collections import deque

def find_dead_ends(file_path):
    """
    Find all dead-end pages in the graph and return their list in removal order.

    Parameters:
        file_path (str): Path to the dataset file.

    Returns:
        list: List of dead-end pages in their removal order.
    """
    # Parse the graph
    in_neighbors, out_neighbors = parse_graph(file_path)

    # Initialize out-degree array
    D = {node: len(out_neighbors.get(node, [])) for node in set(in_neighbors) | set(out_neighbors)}

    # Identify initial dead-ends
    q = deque([node for node, degree in D.items() if degree == 0])

    # Recursive elimination of dead-ends
    dead_ends = []
    visited = set()  # To ensure nodes are processed only once
    while q:
        node = q.popleft()
        if node not in visited:
            visited.add(node)
            dead_ends.append(node)
            for in_neighbor in in_neighbors.get(node, []):
                D[in_neighbor] -= 1
                if D[in_neighbor] == 0:
                    q.append(in_neighbor)

    return dead_ends

dead_ends = find_dead_ends(file_path)
print("Dead-End Pages (in Removal Order):", dead_ends)

Dead-End Pages (in Removal Order): [24584, 65551, 258066, 208925, 213021, 237599, 28703, 131107, 53286, 139311, 20530, 122938, 90175, 69696, 32833, 229441, 200770, 81987, 200772, 229449, 76, 114765, 262222, 204879, 118862, 80, 127058, 127062, 86103, 82011, 91, 213089, 163941, 151653, 16492, 108, 143470, 262255, 266351, 16494, 77938, 151673, 262269, 270464, 45184, 200832, 233602, 28809, 110731, 37003, 73872, 225425, 61586, 155801, 114844, 188574, 168094, 147616, 250016, 41122, 221346, 266402, 155811, 77992, 196782, 77999, 200890, 274619, 155849, 78030, 159951, 192719, 245969, 73939, 176342, 213209, 192738, 65766, 225511, 78056, 123114, 20715, 164079, 94452, 106740, 184565, 90359, 168189, 123134, 213249, 135428, 24838, 225552, 98577, 86291, 78103, 61721, 192798, 241952, 123171, 176422, 258348, 127278, 250159, 49456, 94512, 114994, 106804, 250164, 311, 160056, 57657, 213303, 147768, 86333, 49476, 217423, 242000, 49489, 106843, 164186, 78181, 135534, 24944, 94578, 156020, 115060, 12663, 13

# Exercise 5 (in the PDF)

# Exercise 6

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, TimestampType, StringType

def load_spot_instance_prices(file_name):
    """
    Load a .gz file with spot instance prices into a Spark DataFrame.
    
    Parameters:
        file_name (str): Path to the compressed .gz file.
    
    Returns:
        pyspark.sql.DataFrame: A Spark DataFrame with the data.
    """
    # Initialize SparkSession
    spark = SparkSession.builder.appName("SpotInstancePricesLoader").getOrCreate()
    
    # Define schema with memory-efficient data types
    schema = StructType([
        StructField("Type", StringType(), True),  # Drop this column later
        StructField("Price", DoubleType(), True),
        StructField("Timestamp", TimestampType(), True),
        StructField("InstanceType", StringType(), True),
        StructField("ProductDescription", StringType(), True),
        StructField("AvailabilityZone", StringType(), True)
    ])
    
    # Read the .gz file into a DataFrame
    df = spark.read.csv(file_name, sep="\t", schema=schema, header=True)
    
    # Drop the "Type" column
    df = df.drop("Type")
    
    return df

# Example usage
# File path to test
file_name = "dataset-EC2-series/prices-ca-central-1-2019-05-24.txt.gz"

# Load the DataFrame
df = load_spot_instance_prices(file_name)

# Calculate the average price per combination of InstanceType and ProductDescription
avg_prices = (
    df.groupBy("InstanceType", "ProductDescription")
        .avg("Price")
        .withColumnRenamed("avg(Price)", "AveragePrice")
)

# Show the results
avg_prices.show()


+------------+------------------+--------------------+
|InstanceType|ProductDescription|        AveragePrice|
+------------+------------------+--------------------+
|   c4.xlarge|           Windows| 0.23910000000000034|
|    t3.micro|           Windows|0.012699999999999993|
|  t3.2xlarge|           Windows| 0.25780000000000025|
|  t2.2xlarge|           Windows|  0.1848999999999999|
|  m5.4xlarge|        Linux/UNIX| 0.25776746268656775|
|   t2.xlarge|        Linux/UNIX|  0.0614000000000001|
| m5.12xlarge|        Linux/UNIX|  1.2112850490196077|
|   i3.xlarge|           Windows| 0.28719999999999923|
| x1.16xlarge|           Windows|   5.144800000000003|
|   c5.xlarge|        Linux/UNIX| 0.05872099644128116|
|  c5.9xlarge|        Linux/UNIX|  0.6711771978021982|
|    t2.micro|           Windows|0.008399999999999982|
|m5d.12xlarge|           Windows|   2.936500000000008|
|    t2.small|        Linux/UNIX|0.007700000000000022|
|  c5.4xlarge|           Windows|  0.9672999999999973|
| c5.18xla